In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pprint
import pandas as pd
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import sys

## *Garage

In [20]:
# WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located(
#             (By.CSS_SELECTOR, ".set_sido_cd_btn"))
#     )

## *init

In [21]:
driver_path = '/Users/timothychun/Downloads/chromedriver'

f = open("/Users/timothychun/Desktop/grade.txt","w",encoding = 'utf-8')

driver = webdriver.Chrome(driver_path)

#셀(cell)을 execute하기 위한 클래스
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

### URL 불러오기

In [22]:
url = "https://www.ebsi.co.kr/ebs/xip/xipa/retrievePastGrdCutWrongAnswerRate.ebs"

driver.get(url)

# "역대 등급컷/오답률 TOP15" 라는 title이 나올때까지 기다리기
WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "body > div.wrap > section > div > div.content > div.title_wrap2 > h3"))
    )

<selenium.webdriver.remote.webelement.WebElement (session="b621ee4c7dfa485f76d3c50aa3fd2b12", element="345e8910-1127-484b-9bcc-c812e62367b2")>

In [23]:
# 연도와 선택해야하는 월
years = [2019,2018,2017,2016]
months = [6,9,11]

In [24]:
def get_jiktam_by_year(year):
    
    #년도 선택하기
    time.sleep(2)
    grade_select = Select(driver.find_element_by_css_selector(f"#gYear"))
    try:
        grade_select.select_by_visible_text(f'{year}년')
    except NoSuchElementException: #만약 선택자가 선택한 월이 존재하지 않때
        print("그런 년도가 없습니다!")
        driver.close()
        
    #학년 선택하기
    time.sleep(2)
    year_select = Select(driver.find_element_by_css_selector("#gStdntGrd"))
    year_select.select_by_visible_text('3학년')
    
    f.write(f"----------{year}---------\n")

    #직탐을 치는 월 
    time.sleep(2)
    for month in months:
        f.write(f"\n~~~{month}~~~\n")
        target_year_month_id = Select(driver.find_element_by_css_selector("#gMonth"))
        if month<10:
            Month_text = "0"+ str(month)
        else:
            Month_text = str(month)
        try:
            target_year_month_id.select_by_visible_text(f'{Month_text}월')
        except NoSuchElementException: #만약 선택자가 선택한 월이 존재하지 않때
            pass
        
        #검색버튼 누르기!
        time.sleep(1)
        driver.find_element_by_css_selector('#pastGrdCutArea > div.search_box > div > button').click()
        time.sleep(2)

        #직탐 선택하기
        driver.find_element_by_css_selector("#arBtn4 > a").click()
        time.sleep(2)
        html=driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        a = soup.find_all(class_="table_scroll2")

        grades = []
        # subjects = ["농업 기초 기술","공업 일반","상업 경제","수산·해운 산업 기초","인간 발달","성공적인 직업 생활"]
        subjects = ["농업 이해","농업 기초 기술","공업 일반","기초 제도","상업 경제","회계 원리","해양의 이해","수산·해운 산업 기초","인간 발달","생활 서비스 산업의 이해"]
        i=0
        for div in a:
            tr = div.find_all('tr')
            d = 0
            for td in tr:
                level = td.find(scope = 'row')
                grade = td.find(class_="c_blue")
                if level == None or grade == None and not d:
                    grades.append(subjects[i])
                    i+=1
                    continue
                elif level == None or grade == None:
                    continue
                level = td.find(scope = 'row').get_text()
                if level == 9:
                    d=0
                else:
                    d+=1
                grade = td.find(class_="c_blue").get_text()
                
                grades.append(dict(등급=level,점수=grade))
        farm = grades[grades.index('농업 기초 기술')+1:grades.index('공업 일반')]
        industry = grades[grades.index('공업 일반')+1:grades.index('기초 제도')]
        econ = grades[grades.index('상업 경제')+1:grades.index('회계 원리')]
        fisheries = grades[grades.index('수산·해운 산업 기초')+1:grades.index('인간 발달')]
        human = grades[grades.index('인간 발달')+1:grades.index('생활 서비스 산업의 이해')]
        # job = grades[grades.index('성공적인 직업 생활')+1:]

        f.write("\n_농업 기초 기술_\n")
        for g in farm:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")

        f.write("\n_공업 일반_\n")
        for g in industry:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")

        f.write("\n_상업 경제_\n")
        for g in econ:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")
        
        f.write("\n_수산·해운 산업 기초_\n")
        for g in fisheries:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")
        
        f.write("\n_인간 발달_\n")
        for g in human:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")

        # f.write("\n_성공적인 직업 생활_\n")
        # for g in job:
        #     for k,v in g.items():
        #         f.write(f"{k}:{v}\n")

    f.write("\n\n")




for i in years:
    year = i
    get_jiktam_by_year(year)


f.close()
    

In [25]:
#2021 성직만 가져오기
f = open("/Users/timothychun/Desktop/seung_jik.txt","w",encoding = 'utf-8')
def sueng_jik():
    year = 2021
    months = [6,9]
    #년도 선택하기
    time.sleep(2)
    grade_select = Select(driver.find_element_by_css_selector(f"#gYear"))
    try:
        grade_select.select_by_visible_text(f'{year}년')
    except NoSuchElementException: #만약 선택자가 선택한 월이 존재하지 않때
        print("그런 년도가 없습니다!")
        driver.close()
        
    #학년 선택하기
    time.sleep(2)
    year_select = Select(driver.find_element_by_css_selector("#gStdntGrd"))
    year_select.select_by_visible_text('3학년')
    
    f.write(f"----------{year}---------\n")

    #직탐을 치는 월 
    time.sleep(2)
    for month in months:
        f.write(f"\n~~~{month}~~~\n")
        target_year_month_id = Select(driver.find_element_by_css_selector("#gMonth"))
        if month<10:
            Month_text = "0"+ str(month)
        else:
            Month_text = str(month)
        try:
            target_year_month_id.select_by_visible_text(f'{Month_text}월')
        except NoSuchElementException: #만약 선택자가 선택한 월이 존재하지 않때
            pass
        
        #검색버튼 누르기!
        time.sleep(1)
        driver.find_element_by_css_selector('#pastGrdCutArea > div.search_box > div > button').click()
        time.sleep(2)

        #직탐 선택하기
        driver.find_element_by_css_selector("#arBtn4 > a").click()
        time.sleep(2)
        html=driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        a = soup.find_all(class_="table_scroll2")

        grades = []
        subjects = ["농업 기초 기술","공업 일반","상업 경제","수산·해운 산업 기초","인간 발달","성공적인 직업 생활"]
        # subjects = ["농업 이해","농업 기초 기술","공업 일반","기초 제도","상업 경제","회계 원리","해양의 이해","수산·해운 산업 기초","인간 발달","생활 서비스 산업의 이해"]
        i=0
        for div in a:
            tr = div.find_all('tr')
            d = 0
            for td in tr:
                level = td.find(scope = 'row')
                grade = td.find(class_="c_blue")
                if level == None or grade == None and not d:
                    grades.append(subjects[i])
                    i+=1
                    continue
                elif level == None or grade == None:
                    continue
                level = td.find(scope = 'row').get_text()
                if level == 9:
                    d=0
                else:
                    d+=1
                grade = td.find(class_="c_blue").get_text()
                
                grades.append(dict(등급=level,점수=grade))
        farm = grades[grades.index('농업 기초 기술')+1:grades.index('공업 일반')]
        industry = grades[grades.index('공업 일반')+1:grades.index('상업 경제')]
        econ = grades[grades.index('상업 경제')+1:grades.index('수산·해운 산업 기초')]
        fisheries = grades[grades.index('수산·해운 산업 기초')+1:grades.index('인간 발달')]
        human = grades[grades.index('인간 발달')+1:grades.index('성공적인 직업 생활')]
        job = grades[grades.index('성공적인 직업 생활')+1:]

        f.write("\n_농업 기초 기술_\n")
        for g in farm:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")

        f.write("\n_공업 일반_\n")
        for g in industry:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")

        f.write("\n_상업 경제_\n")
        for g in econ:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")
        
        f.write("\n_수산·해운 산업 기초_\n")
        for g in fisheries:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")
        
        f.write("\n_인간 발달_\n")
        for g in human:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")

        f.write("\n_성공적인 직업 생활_\n")
        for g in job:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")

    f.write("\n\n")


sueng_jik()
f.close()

In [26]:
f = open("/Users/timothychun/Desktop/fifteen_to_end.txt","w",encoding = 'utf-8')
def fifteen_to_end(year):
    months = [6,9,11]
    
    #년도 선택하기
    time.sleep(2)
    grade_select = Select(driver.find_element_by_css_selector(f"#gYear"))
    try:
        grade_select.select_by_visible_text(f'{year}년')
    except NoSuchElementException: #만약 선택자가 선택한 월이 존재하지 않때
        print("그런 년도가 없습니다!")
        driver.close()
        
    #학년 선택하기
    time.sleep(2)
    year_select = Select(driver.find_element_by_css_selector("#gStdntGrd"))
    year_select.select_by_visible_text('3학년')
    
    f.write(f"----------{year}---------\n")

    #직탐을 치는 월 
    time.sleep(2)
    for month in months:
        f.write(f"\n~~~{month}~~~\n")
        target_year_month_id = Select(driver.find_element_by_css_selector("#gMonth"))
        if month<10:
            Month_text = "0"+ str(month)
        else:
            Month_text = str(month)
        try:
            target_year_month_id.select_by_visible_text(f'{Month_text}월')
        except NoSuchElementException: #만약 선택자가 선택한 월이 존재하지 않때
            pass
        
        #검색버튼 누르기!
        time.sleep(1)
        driver.find_element_by_css_selector('#pastGrdCutArea > div.search_box > div > button').click()
        time.sleep(2)

        #직탐 선택하기
        driver.find_element_by_css_selector("#arBtn4 > a").click()
        time.sleep(2)
        html=driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        a = soup.find_all(class_="table_scroll2")

        grades = []
        subjects = ['농생명산업','공업','상업정보','수산해운','가사실업']
        i=0
        for div in a:
            tr = div.find_all('tr')
            d = 0
            for td in tr:
                level = td.find(scope = 'row')
                grade = td.find(class_="c_blue")
                if level == None or grade == None and not d:
                    grades.append(subjects[i])
                    i+=1
                    continue
                elif level == None or grade == None:
                    continue
                level = td.find(scope = 'row').get_text()
                if level == 9:
                    d=0
                else:
                    d+=1
                grade = td.find(class_="c_blue").get_text()
                
                grades.append(dict(등급=level,점수=grade))
        industry = grades[grades.index('공업')+1:grades.index('상업정보')]
        commerce = grades[grades.index('상업정보')+1:grades.index('수산해운')]

        f.write("\n_공업_\n")
        for g in industry:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")
        
        f.write("\n_상업정보_\n")
        for g in commerce:
            for k,v in g.items():
                f.write(f"{k}:{v}\n")

        

    f.write("\n\n")



years = [2015,2014,2013]
for i in years:
    year = i
    fifteen_to_end(year)

f.close()
    

ValueError: '회계 원리' is not in list

In [ ]:
driver.close()